In [1]:
import pandas as pd
import requests
import time
from calendar import monthrange # '말일'을 계산하기 위해 추가
from tqdm import tqdm

# --- 1. 사용자 설정 ---
INPUT_CSV = 'train_data_with_elevation.csv'
OUTPUT_CSV = 'train_data_final_with_fault_counts_past_year_full_month.csv'
PROCESS_LIMIT = None # ⚠️ 테스트를 위해 10으로 유지 (성공 시 None으로 변경)

# --- 2. API 요청 및 단층 분석 함수 (V14 - '과거 1년 + 현재 달' 카운트) ---
def get_fault_counts_from_usgs(row):
    """
    (수정됨) 1단계 검색에서 '과거 1년 ~ 현재 달 말일'의 모든 지진 단층 타입을 카운트합니다.
    """
    horizontal_count = 0
    vertical_count = 0

    try:
        # === 1단계: '검색' (수정됨: 과거 1년 + 현재 달) ===
        year = int(row['Year'])
        month = int(row['Month'])

        # (수정) 끝나는 날짜: 이 지진이 발생한 달의 '말일'
        _, last_day = monthrange(year, month)
        endtime = f"{year}-{month:02d}-{last_day}" # 예: 2022-11-30

        # (유지) 시작 날짜: 1년 전 해당 월의 1일
        starttime = f"{year - 1}-{month:02d}-01" # 예: 2021-11-01

        search_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
        params = {
            'format': 'geojson',
            'starttime': starttime, # V14: 1년 전
            'endtime': endtime,     # V14: 현재 달 말일
            'latitude': row['latitude'],
            'longitude': row['longitude'],
            'maxradiuskm': 200,  # 반경 200km
            'minmagnitude': 6.0  # M 6.0 이상
        }

        response = requests.get(search_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        found_quakes = data.get('features', [])

        if not found_quakes:
            return pd.Series([0, 0]) # "Not Found"

        # === 2단계: (V13과 동일) '모든' 찾은 지진을 순회하며 'rake' 파싱 ===

        for quake in found_quakes:
            detail_url = quake['properties'].get('detail')
            if not detail_url:
                continue

            time.sleep(0.1)
            detail_response = requests.get(detail_url, timeout=10)
            detail_response.raise_for_status()
            detail_data = detail_response.json()

            products = detail_data['properties'].get('products', {})
            rake_value = None

            all_products = products.get('moment-tensor', []) + products.get('focal-mechanism', [])
            if not all_products:
                 continue

            best_product = None
            for p in all_products:
                if 'gcmt' in p.get('id','').lower(): best_product = p; break
            if best_product is None:
                for p in all_products:
                     if 'us' in p.get('id','').lower() or p.get('code','').lower() == 'us': best_product = p; break
            if best_product is None:
                best_product = all_products[0]

            if best_product:
                props = best_product.get('properties', {})
                rake_str = props.get('nodal-plane-1-rake')
                if rake_str is None: rake_str = props.get('rake')
                if rake_str is not None: rake_value = float(rake_str)

            if rake_value is not None:
                if (45 <= rake_value <= 135) or (-135 <= rake_value <= -45):
                    vertical_count += 1
                else:
                    horizontal_count += 1

        return pd.Series([horizontal_count, vertical_count])

    except Exception as e:
        # print(f"  -> Error processing row {row.name}: {e}")
        return pd.Series([0, 0])
    finally:
        time.sleep(0.1)

# --- 3. 메인 스크립트 실행 (수정됨) ---

def main():
    print(f"'{INPUT_CSV}' 파일 읽는 중...")
    try:
        df = pd.read_csv(INPUT_CSV)
    except FileNotFoundError:
        print(f"오류: '{INPUT_CSV}' 파일을 찾을 수 없습니다.")
        return

    if PROCESS_LIMIT is not None:
        print(f"--- 테스트 모드: 처음 {PROCESS_LIMIT}개 행만 처리합니다. (V14 최종본) ---")
        df_to_process = df.iloc[:PROCESS_LIMIT].copy()
    else:
        print("--- 전체 데이터 처리 모드 ---")
        df_to_process = df.copy()

    tqdm.pandas(desc="USGS V14 'Past 1 Year + Current Month' 카운트 가져오는 중")

    new_cols_df = df_to_process.progress_apply(get_fault_counts_from_usgs, axis=1)

    # (수정됨) 컬럼 이름 변경
    new_cols_df.columns = ['horizontal_count_1y_full', 'vertical_count_1y_full']

    df_to_process = pd.concat([df_to_process, new_cols_df], axis=1)

    print("\n--- API 요청 완료 ---")

    print("\n분석 결과 미리보기 ('horizontal_count_1y_full', 'vertical_count_1y_full' 열 추가):")
    print(df_to_process[['magnitude', 'tsunami', 'horizontal_count_1y_full', 'vertical_count_1y_full']].head(10))

    output_file = f"TEST_{OUTPUT_CSV}"
    print(f"\n테스트 결과가 '{output_file}' 파일로 저장되었습니다.")
    df_to_process.to_csv(output_file, index=False)

    print("\n--- 'horizontal_count_1y_full' (수평이동 횟수) 결과 요약 ---")
    print(pd.crosstab(df_to_process['horizontal_count_1y_full'], df_to_process['tsunami']))

    print("\n--- 'vertical_count_1y_full' (수직이동 횟수) 결과 요약 ---")
    print(pd.crosstab(df_to_process['vertical_count_1y_full'], df_to_process['tsunami']))

if __name__ == "__main__":
    main()

'train_data_with_elevation.csv' 파일 읽는 중...
--- 전체 데이터 처리 모드 ---


USGS V14 'Past 1 Year + Current Month' 카운트 가져오는 중: 100%|██████████| 782/782 [1:11:04<00:00,  5.45s/it]


--- API 요청 완료 ---

분석 결과 미리보기 ('horizontal_count_1y_full', 'vertical_count_1y_full' 열 추가):
   magnitude  tsunami  horizontal_count_1y_full  vertical_count_1y_full
0        7.0        1                         1                       1
1        6.9        0                         0                       1
2        7.0        1                         1                       1
3        7.3        1                         0                       1
4        6.6        1                         3                       2
5        7.0        1                         3                       2
6        6.8        1                         3                       2
7        6.7        1                         0                       1
8        6.8        1                         0                       2
9        7.6        1                         0                       2

테스트 결과가 'TEST_train_data_final_with_fault_counts_past_year_full_month.csv' 파일로 저장되었습니다.

--- 'horizontal_count_1y_f

In [28]:
import pandas as pd
import requests
import json
import time
from calendar import monthrange

# --- 1. 설정 ---
INPUT_CSV = 'train_data_with_elevation.csv'

# --- 2. 데이터 읽기 (첫 번째 행만) ---
try:
    df = pd.read_csv(INPUT_CSV)
    first_row = df.iloc[6] # Row 0
except FileNotFoundError:
    print(f"오류: '{INPUT_CSV}' 파일을 찾을 수 없습니다.")
    exit()

print(f"--- 1번 행(Row 0) 데이터로 디버깅 시작 ---")
print(f"데이터: M{first_row['magnitude']}, {first_row['Year']}-{first_row['Month']}, ({first_row['latitude']}, {first_row['longitude']})")

# --- 3. 1단계: 검색 (Search) ---

is_horizontal = 0
is_vertical = 0

try:
    # 3-1. 검색 파라미터 설정
    year = int(first_row['Year'])
    month = int(first_row['Month'])
    _, last_day = monthrange(year, month)

    starttime = f"{year}-{month:02d}-01"
    endtime = f"{year}-{month:02d}-{last_day}"

    search_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    params = {
        'format': 'geojson',
        'starttime': starttime,
        'endtime': endtime,
        'latitude': first_row['latitude'],
        'longitude': first_row['longitude'],
        'maxradiuskm': 200,  # 반경 200km
        'minmagnitude': 6.0  # M 6.0 이상
        # 'includeallproperties' 제거 (사용자님의 'count: 2' 성공 사례 기반)
    }

    # 3-2. 1단계 API 요청
    print("\n[1단계: 검색] API 요청을 보냅니다...")
    response = requests.get(search_url, params=params, timeout=10)

    # ★★★ (요청 1) 1단계 검색 URL 출력 ★★★
    print(f"\n[요청 URL 1 (검색)]\n{response.request.url}\n")
    # ★★★★★★★★★★★★★★★★★★★★★★★★★★★

    response.raise_for_status()
    data = response.json()

    found_quakes = data.get('features', [])

    if not found_quakes:
        print("[1단계: 실패] 이 조건으로 지진을 찾지 못했습니다. (0개 반환)")
        exit()

    print(f"[1단계: 성공] 총 {len(found_quakes)}개의 지진을 찾았습니다.")

    # --- 4. 2단계: 상세 정보 (Detail) ---

    print("\n[2단계: 상세] 찾은 지진들에 대해 2차 접속을 시도합니다...")

    for i, quake in enumerate(found_quakes):
        print("-" * 30)
        print(f"  [ {i+1} / {len(found_quakes)} 번째 지진 ]")

        detail_url = quake['properties'].get('detail')

        if not detail_url:
            print("  -> 'detail' URL이 없어 파싱에 실패했습니다.")
            continue

        # ★★★ (요청 2) 2단계 상세 URL 출력 ★★★
        print(f"  [요청 URL 2 (상세)]\n  {detail_url}\n")
        # ★★★★★★★★★★★★★★★★★★★★★★★★★★★

        try:
            # 2-1. 상세 정보 페이지로 2차 접속
            time.sleep(0.1)
            detail_response = requests.get(detail_url, timeout=10)
            detail_response.raise_for_status()
            detail_data = detail_response.json()

            # 2-2. 'products' 객체 파싱
            products = detail_data['properties'].get('products', {})

            focal_mech = products.get('focal-mechanism')
            moment_tensor = products.get('moment-tensor')

            fault_type_str = ""

            if focal_mech and len(focal_mech) > 0 and 'properties' in focal_mech[0]:
                fault_type_str = focal_mech[0]['properties'].get('focal-mechanism-type', '').upper()

            if not fault_type_str and moment_tensor and len(moment_tensor) > 0 and 'properties' in moment_tensor[0]:
                fault_type_str = moment_tensor[0]['properties'].get('tensor-type', '').upper()

            # 2-3. '수직'/'수평'으로 분류
            if "STRIKE-SLIP" in fault_type_str or "STRIKE_SLIP" in fault_type_str:
                is_horizontal = 1
                print("  [결과] 파싱 성공: 'Horizontal' (수평)")
            elif "REVERSE" in fault_type_str or "THRUST" in fault_type_str:
                is_vertical = 1
                print("  [결과] 파싱 성공: 'Vertical' (수직)")
            elif "NORMAL" in fault_type_str:
                is_vertical = 1
                print("  [결과] 파싱 성공: 'Vertical' (수직)")
            else:
                print("  [결과] 파싱 실패: 'products'는 찾았으나, 아는 단층 타입이 없음")
                # (디버깅) products 객체 전체를 출력해서 확인
                # print(json.dumps(products, indent=2))

        except Exception as e_detail:
            print(f"  -> 2단계 접속 오류: {e_detail}")

except Exception as e_search:
    print(f"\n[1단계: 오류] {e_search}")

print("-" * 30)
print("\n--- 최종 디버깅 결과 (Row 0) ---")
print(f"is_horizontal: {is_horizontal}")
print(f"is_vertical: {is_vertical}")

--- 1번 행(Row 0) 데이터로 디버깅 시작 ---
데이터: M6.8, 2022-11, (-25.9678, 178.363)

[1단계: 검색] API 요청을 보냅니다...

[요청 URL 1 (검색)]
https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-11-01&endtime=2022-11-30&latitude=-25.9678&longitude=178.363&maxradiuskm=200&minmagnitude=6.0

[1단계: 성공] 총 4개의 지진을 찾았습니다.

[2단계: 상세] 찾은 지진들에 대해 2차 접속을 시도합니다...
------------------------------
  [ 1 / 4 번째 지진 ]
  [요청 URL 2 (상세)]
  https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000ipkb&format=geojson

  [결과] 파싱 실패: 'products'는 찾았으나, 아는 단층 타입이 없음
------------------------------
  [ 2 / 4 번째 지진 ]
  [요청 URL 2 (상세)]
  https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000inht&format=geojson

  [결과] 파싱 실패: 'products'는 찾았으나, 아는 단층 타입이 없음
------------------------------
  [ 3 / 4 번째 지진 ]
  [요청 URL 2 (상세)]
  https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000ingi&format=geojson

  [결과] 파싱 실패: 'products'는 찾았으나, 아는 단층 타입이 없음
------------------------------
  [ 4 / 4 번째 지진 ]
  